Dataset recap


Total: 17,898 examples

Positive (Pulsars): 1,639 (~9.2%)

Negative (RFI/noise): 16,259 (~90.8%)


Features (all continuous):

    Mean of integrated profile

    Std dev of integrated profile

    Excess kurtosis of integrated profile

    Skewness of integrated profile

    Mean of DM-SNR curve

    Std dev of DM-SNR curve

    Excess kurtosis of DM-SNR curve

    Skewness of DM-SNR curve

Labels: 0 (negative), 1 (positive)

https://archive.ics.uci.edu/dataset/372/htru2

In [16]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


In [17]:
# Load the CSV file, skipping header if present
data = np.loadtxt('HTRU_2.csv', delimiter=',')  # skiprows=1 if there is a header

# Split into features and labels
X = data[:, :-1]  # all columns except last
y = data[:, -1]   # last column

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print(np.unique(y, return_counts=True))  # Check distribution of classes

Features shape: (17898, 8)
Labels shape: (17898,)
(array([0., 1.]), array([16259,  1639]))


In [18]:
# Define function for splitting the training and testing data


def split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    # Find the indices for each class
    classes = np.unique(y)
    train_indices = []
    test_indices = []

    for cls in classes:
        cls_indices = np.where(y == cls)[0]
        np.random.shuffle(cls_indices)

        n_test = int(len(cls_indices) * test_size)

        test_indices.extend(cls_indices[:n_test])
        train_indices.extend(cls_indices[n_test:])

    # Convert to numpy arrays
    train_indices = np.array(train_indices)
    test_indices = np.array(test_indices)

    # Shuffle again to mix classes in final splits
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)

    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]

    return X_train, X_test, y_train, y_test


In [19]:
# Define training and testing data
X_train, X_test, y_train, y_test = split(X,y)

In [20]:
print('First 5 elements: ', X_train[:5])
print(type(X_train))

First 5 elements:  [[ 1.04445312e+02  5.47847790e+01  3.28619322e-01 -3.47524523e-01
   1.40719064e+00  1.48152135e+01  1.17013504e+01  1.49252476e+02]
 [ 9.16250000e+01  4.75821658e+01  8.44691022e-01  1.32428500e+00
   1.06688963e+00  1.12127737e+01  1.51077482e+01  2.75215672e+02]
 [ 1.25632812e+02  3.96115654e+01  2.30649882e-01  8.62036047e-01
   1.88545151e+00  1.42206485e+01  1.12309687e+01  1.58386793e+02]
 [ 1.01585938e+02  4.67056152e+01  6.03978802e-01  1.12749467e+00
   2.26337793e+00  1.35314583e+01  9.52258626e+00  1.27246241e+02]
 [ 1.35468750e+02  4.95559875e+01  7.36776720e-02 -2.72905389e-01
   3.21655518e+00  2.19859888e+01  8.13866035e+00  7.18727605e+01]]
<class 'numpy.ndarray'>


In [21]:
print("First five elements in y_train are:\n", y_train[:5])
print("Type of y_train:",type(y_train))

First five elements in y_train are:
 [0. 0. 0. 0. 0.]
Type of y_train: <class 'numpy.ndarray'>


In [22]:
print ('The shape of X_train is: ' + str(X_train.shape))
print ('The shape of y_train is: ' + str(y_train.shape))
print ('We have m = %d training examples' % (len(y_train)))

The shape of X_train is: (14320, 8)
The shape of y_train is: (14320,)
We have m = 14320 training examples


In [23]:
# Sigmoid Function 

def sigmoid(z):
    g = 1/(1+np.exp(-1*z))

    return g


In [40]:
# COST FUNCTION FOR LOGISTIC REGRESSION

def cost_function(X,y,w,b, *argv):

    m,n = X.shape

    L = 0

    for i in range(0,m):
        z = np.dot(w,X[i]) + b
        f = sigmoid(z)

        if y[i] == 1:
            L -= np.log(f)
        else:
            L-= np.log(1-f)

    total_cost = L/m 

    return total_cost

In [26]:
# GRADIENT DESCENT 

def gradient(X,y,w,b,*argv):  #argv is for in case i add regularization

    m,n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    for i in range(m):
        z = np.dot(w,X[i]) + b
        f = sigmoid(z)

        dj_db_i = f - y[i]
        dj_db += dj_db_i

        for j in range(n):
            dj_dw[j] += (f-y[i])*X[i][j]

    dj_dw = dj_dw/m
    dj_db = dj_db/m

    return dj_db, dj_dw


In [ ]:
def gradient_descent(X,y,w_in,b_in,cost_function, gradient_function, alpha, num_iters, lambda_=0.1): #Regularization will be added later

    m = len(X)
    
    for i in range(num_iters):

        dj_db, dj_dw = gradient_function(X,y,w_in,b_in)

        w_in = w_in - alpha*dj_dw
        b_in = b_in - alpha*dj_db

        '''if i<100000:
            c = cost_function(X,y,w_in,b_in)'''
            

    return w_in,b_in

In [28]:
# Predicting Pulsars

def predict(X,w,b):

    m,n = X.shape
    p = np.zeros(m)

    for i in range(m):
        z = np.dot(w,X[i]) + b
        f = sigmoid(z)
        if f >= 0.5:
            p[i] = 1
    
    return p

In [ ]:
# Put the values and implement logistic regression
n = X_train.shape[1]
w = np.zeros(n)
b = 0

w, b = gradient_descent(X_train, y_train, w, b, cost_function, gradient, alpha=0.01, num_iters=1000)


In [43]:
# Calculate accuracy using some metric

y_pred = predict(X_test, w, b)

print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Precision: 0.9139784946236559
Recall: 0.7798165137614679
F1 Score: 0.8415841584158416
ROC AUC: 0.8862170849336407
Confusion Matrix:
 [[3227   24]
 [  72  255]]
